In [ ]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import torch
import random
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms, models
from PIL import Image
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from sklearn.utils import class_weight
from torch.utils.data import WeightedRandomSampler
from tqdm import tqdm
from google.colab import files

In [ ]:
# Set a fixed random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
class CustomHFDataset(Dataset):
    def __init__(self, hf_dataset, transform=None, test_flag=False):
        """
        Args:
            hf_dataset: A split of the Hugging Face dataset (e.g., train, val, test)
            transform: PyTorch transforms to apply on the images
        """
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.test_flag = test_flag

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # Get the raw example from the Hugging Face dataset
        example = self.hf_dataset[idx]
        image = example['image']
        label = example['label']

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        if self.test_flag:
            return image, example['idx']
        else:
            return image, label

# Load dataset
dataset = load_dataset('hmdliu/ACAC-4K')

# Utilize the precomputed mean and std
dataset_mean = [0.620266854763031, 0.5429694056510925, 0.4473448693752289]
dataset_std = [0.12819308042526245, 0.11423765122890472, 0.09967498481273651]

# Define transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),  # Ensure images are resized to 512x512
    transforms.ToTensor(),
    transforms.Normalize(
        mean=dataset_mean,
        std=dataset_std
    )
])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/719 [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/466M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
class BinaryHFDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        """
        Converts multi-class labels to binary labels: AI (1) and NonAI (0)
        """
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        example = self.hf_dataset[idx]
        image = example['image']
        label = example['label']

        # Define your mapping here
        # Example mapping: labels 0,1,2,3,4 -> NonAI (0); labels 5 -> AI (1)
        if label in [0, 1, 2, 3, 4]:
            binary_label = 0  # NonAI
        else:
            binary_label = 1  # AI

        if self.transform:
            image = self.transform(image)

        return image, binary_label

### Train the initial AI detector model



In [ ]:
# Define enhanced data augmentation transforms
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=dataset_mean, std=dataset_std)
])

# Initialize the binary dataset
binary_train_dataset = BinaryHFDataset(dataset['train'], transform=transform)

# Define the split sizes
train_size = int(0.8 * len(binary_train_dataset))
val_size = len(binary_train_dataset) - train_size

# Perform the split
train_dataset, val_dataset = random_split(binary_train_dataset, [train_size, val_size],
                                        generator=torch.Generator().manual_seed(SEED))

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# Extract labels for class weight computation
labels = [binary_train_dataset[i][1] for i in train_dataset.indices]


# Compute class weights
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(labels),
                                                  y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Apply WeightedRandomSampler to handle class imbalance
samples_weights = [class_weights[label] for label in labels]
samples_weights = torch.tensor(samples_weights, dtype=torch.float)
sampler = WeightedRandomSampler(weights=samples_weights,
                                num_samples=len(samples_weights),
                                replacement=True)

Training samples: 2560
Validation samples: 640


In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
# Initialize ResNet (without pretraining)
model = models.resnet18(pretrained=False)

# Modify the final fully connected layer
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 1)
)  # Output a single logit for binary classification

# Move the model to the device
model = model.to(device)

# Define Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

criterion = FocalLoss(alpha=1, gamma=2)

# Define optimizer with weight decay
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Define learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
num_epochs = 15  # Adjust as needed

best_val_f1 = 0.0
patience = 10
trigger_times = 0

# To store predictions and labels
all_train_preds = []
all_train_labels = []
all_train_indices = []

for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation Phase
    model.eval()
    val_running_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)

            preds = torch.sigmoid(outputs)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss = val_running_loss / len(val_loader.dataset)

    # Convert predictions to binary
    binary_preds = [1 if p >= 0.5 else 0 for p in all_preds]
    binary_labels = [int(l) for l in all_labels]

    # Calculate metrics
    acc = accuracy_score(binary_labels, binary_preds)
    f1 = f1_score(binary_labels, binary_preds)
    cm = confusion_matrix(binary_labels, binary_preds)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Train Loss: {epoch_loss:.4f} | Val Loss: {val_loss:.4f} | "
          f"Accuracy: {acc:.4f} | F1 Score: {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)

    # Early Stopping Check
    if f1 > best_val_f1:
        best_val_f1 = f1
        trigger_times = 0
        # Save the best model
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    # Step the scheduler
    scheduler.step()

<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


Epoch [1/15]
Train Loss: 0.1241 | Val Loss: 0.1172 | Accuracy: 0.7766 | F1 Score: 0.5402
Confusion Matrix:
[[413 140]
 [  3  84]]


<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


Epoch [2/15]
Train Loss: 0.1003 | Val Loss: 0.0928 | Accuracy: 0.8438 | F1 Score: 0.6324
Confusion Matrix:
[[454  99]
 [  1  86]]


<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


...

<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


Epoch [13/15]
Train Loss: 0.0717 | Val Loss: 0.0734 | Accuracy: 0.8625 | F1 Score: 0.6562
Confusion Matrix:
[[468  85]
 [  3  84]]


<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


Epoch [14/15]
Train Loss: 0.0655 | Val Loss: 0.0784 | Accuracy: 0.8625 | F1 Score: 0.6562
Confusion Matrix:
[[468  85]
 [  3  84]]
Epoch [15/15]
Train Loss: 0.0634 | Val Loss: 0.0811 | Accuracy: 0.8625 | F1 Score: 0.6589
Confusion Matrix:
[[467  86]
 [  2  85]]


<ipython-input-13-4a3963f6966a>:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  binary_labels = [int(l) for l in all_labels]


### Perform the first clean

In [ ]:
# Create the original train_dataset
train_dataset = CustomHFDataset(dataset['train'], transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

Load previous model to save time

---



In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load best models
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 1)
)
# Load from Google Drive - update the path to match your Drive structure
model.load_state_dict(torch.load('/content/drive/MyDrive/ML/models/best_model.pth'))
model = model.to(device)

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-8-73aad227ef66>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions

In [ ]:
# model.load_state_dict(torch.load('best_model.pth'))
# model = model.to(device)
# Move the model to evaluation mode
model.eval()

# Ensure no gradients are computed during prediction
torch.no_grad()

# Lists to store original and new labels
original_labels = []
new_labels = []

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Starting AI prediction on the entire training dataset...")

Starting AI prediction on the entire training dataset...


In [ ]:
#############################################
# Step 1: Make predictions and identify top 200 NonAI images to relabel as AI
#############################################

# We'll predict on the entire train dataset using the current binary classifier (model)
# The binary classifier currently expects classes: NonAI(0) and AI(1)
# According to the mapping in the code, original dataset classes 0,1,2,3,4 -> NonAI
# and class 5 -> AI.
#
# We will:
# - Run inference on each image in the train dataset.
# - For each image originally labeled in [0,1,2,3,4] (NonAI), record the predicted probability of AI.
# - Sort these NonAI images by their predicted AI probability (descending).
# - Relabel the top 200 of these as AI.

model.eval()
probabilities = []
indices = []
original_train_labels = []
with torch.no_grad():
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        outputs = model(images)
        # outputs shape: (batch, 1), we can apply sigmoid to get probabilities
        probs = torch.sigmoid(outputs).squeeze().cpu().numpy()

        for j, p in enumerate(probs):
            idx = i*train_loader.batch_size + j
            original_label = train_dataset.hf_dataset[idx]['label']
            probabilities.append((idx, original_label, p))
            original_train_labels.append(original_label)
            indices.append(idx)

# Filter only NonAI images (original labels in [0,1,2,3,4])
nonAI_probs = [(idx, label, p) for (idx, label, p) in probabilities if label in [0,1,2,3,4]]
# Sort by predicted AI probability
nonAI_probs_sorted = sorted(nonAI_probs, key=lambda x: x[2], reverse=True)
# Take top 200
top_200 = nonAI_probs_sorted[:200]

# Create a map of idx to new label for these relabeled images
relabel_map_200 = {x[0]: 5 for x in top_200}  # 5 is AI label

In [ ]:
#############################################
# Prepare a new train dataset with these relabeled images
#############################################

# We'll create a custom dataset class that applies these cleaned labels
class CleanedTrainDataset(Dataset):
    def __init__(self, hf_dataset, transform, relabel_map):
        self.hf_dataset = hf_dataset
        self.transform = transform
        self.relabel_map = relabel_map

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        example = self.hf_dataset[idx]
        image = example['image']
        label = example['label']
        # Apply relabel if exists
        if idx in self.relabel_map:
            label = self.relabel_map[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

cleaned_train_dataset_200 = CleanedTrainDataset(dataset['train'], transform, relabel_map_200)

### Train the AI detector model on the cleaned dataset to improve the performance

In [ ]:
#############################################
# Step 2: Retrain the AI binary classifier with class weights
#############################################

# For binary classifier, we need to split labels into AI/NonAI again
# This time the dataset is "cleaned_train_dataset_200".
# Reuse BinaryHFDataset logic but on top of cleaned_train_dataset_200.

class BinaryFromCleanedDataset(Dataset):
    def __init__(self, cleaned_dataset):
        self.cleaned_dataset = cleaned_dataset

    def __len__(self):
        return len(self.cleaned_dataset)

    def __getitem__(self, idx):
        image, label = self.cleaned_dataset[idx]
        # Binary mapping
        # If label in [0,1,2,3,4] => 0 (NonAI), if label in [5] => 1 (AI)
        binary_label = 1 if label == 5 else 0
        return image, binary_label

binary_train_dataset_200 = BinaryFromCleanedDataset(cleaned_train_dataset_200)

# Split training set into train and val for retraining
train_size = int(0.8 * len(binary_train_dataset_200))
val_size = len(binary_train_dataset_200) - train_size
binary_train_subset_200, binary_val_subset_200 = random_split(binary_train_dataset_200, [train_size, val_size])

train_loader_binary_200 = DataLoader(binary_train_subset_200, batch_size=32, shuffle=True)
val_loader_binary_200 = DataLoader(binary_val_subset_200, batch_size=32, shuffle=False)

# Calculate class weights for binary classification
all_labels_binary_200 = [binary_train_dataset_200[i][1] for i in range(len(binary_train_dataset_200))]
class_wts = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(all_labels_binary_200), y=all_labels_binary_200)
class_wts = torch.tensor(class_wts, dtype=torch.float32).to(device)

# Define binary classifier model (same architecture as before)
binary_model = models.resnet18(pretrained=False)
num_ftrs = binary_model.fc.in_features
binary_model.fc = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 1)
)
binary_model.load_state_dict(torch.load('/content/drive/MyDrive/ML/models/binary_model.pth'))
binary_model = binary_model.to(device)

criterion_binary = nn.BCEWithLogitsLoss(pos_weight=class_wts[1])  # pos_weight for imbalanced binary classes
optimizer_binary = optim.Adam(binary_model.parameters(), lr=1e-4) #Ajust the lr several times

# Train the binary classifier again
epochs = 10
best_val_f1 = 0.0
patience = 10
trigger_times = 0

binary_model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for images, labels in train_loader_binary_200:
        images = images.to(device)
        labels = labels.float().to(device)

        optimizer_binary.zero_grad()
        outputs = binary_model(images)
        loss = criterion_binary(outputs.squeeze(), labels)
        # loss.requires_grad = True
        loss.backward()
        optimizer_binary.step()
        running_loss += loss.item()

    # Validation (optional)
    binary_model.eval()
    val_preds = []
    val_targets = []
    with torch.no_grad():
        val_loss = 0.0
        for images, labels in val_loader_binary_200:
            images = images.to(device)
            labels = labels.float().to(device)
            outputs = binary_model(images)
            loss = criterion_binary(outputs.squeeze(), labels)
            val_loss += loss.item()
            preds = torch.sigmoid(outputs).squeeze().cpu().numpy()
            val_preds.extend((preds > 0.5).astype(int))
            val_targets.extend(labels.cpu().numpy())

    val_acc = accuracy_score(val_targets, val_preds)
    val_f1 = f1_score(val_targets, val_preds)
    print(f"Epoch {epoch+1}/{epochs}: Train Loss: {running_loss/len(train_loader_binary_200):.4f}, Val Loss: {val_loss/len(val_loader_binary_200):.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    # Early Stopping Check
    if val_f1 > best_val_f1 and val_acc <= 0.94: #prevent overfitting
        best_val_f1 = val_f1
        trigger_times = 0
        # Save the best model
        torch.save(model.state_dict(), 'binary_model.pth')
    if val_acc > 0.94: # prevent overfitting
      break
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    binary_model.train()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10: Train Loss: 0.4238, Val Loss: 0.3828, Val Acc: 0.8500, Val F1: 0.6981
Epoch 2/10: Train Loss: 0.2694, Val Loss: 0.2736, Val Acc: 0.9094, Val F1: 0.7914
Epoch 3/10: Train Loss: 0.2528, Val Loss: 0.2673, Val Acc: 0.9172, Val F1: 0.7938
Epoch 4/10: Train Loss: 0.2454, Val Loss: 0.2807, Val Acc: 0.9094, Val F1: 0.7958
Epoch 5/10: Train Loss: 0.2189, Val Loss: 0.3146, Val Acc: 0.9078, Val F1: 0.7958
Epoch 6/10: Train Loss: 0.2115, Val Loss: 0.2694, Val Acc: 0.9125, Val F1: 0.7971
Epoch 7/10: Train Loss: 0.2340, Val Loss: 0.2467, Val Acc: 0.9172, Val F1: 0.8100
Epoch 8/10: Train Loss: 0.2159, Val Loss: 0.2423, Val Acc: 0.9219, Val F1: 0.8162
Epoch 9/10: Train Loss: 0.1942, Val Loss: 0.2863, Val Acc: 0.9141, Val F1: 0.8070
Epoch 10/10: Train Loss: 0.1918, Val Loss: 0.2728, Val Acc: 0.9187, Val F1: 0.8015


### Make a prediction and clean the dataset again with advanced model

In [ ]:
# download checkpoint file
# upload external file before import
from google.colab import files

files.download('binary_model.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create the original train_dataset
train_dataset = CustomHFDataset(dataset['train'], transform=transform)

In [ ]:
#############################################
# Step 3: Perform cleaning again with original dataset, labeling the top 390 NonAI images as AI
#############################################

# We now have a retrained binary classifier (binary_model) that presumably is better.
# Let's run inference again on the ORIGINAL train dataset (not the cleaned one)
# and pick NonAI images by predicted AI probability.

binary_model.eval()
probabilities_round2 = []
with torch.no_grad():
    original_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)
    for i, (images, labels) in enumerate(original_train_loader):
        images = images.to(device)
        outputs = binary_model(images)
        probs = torch.sigmoid(outputs).squeeze().cpu().numpy()

        for j, p in enumerate(probs):
            idx = i*original_train_loader.batch_size + j
            original_label = train_dataset.hf_dataset[idx]['label']
            probabilities_round2.append((idx, original_label, p))

# Filter only NonAI again
nonAI_probs_round2 = [(idx, lbl, p) for (idx, lbl, p) in probabilities_round2 if lbl in [0,1,2,3,4]]

In [ ]:
# The threshold probability is the probability of the 400th highest NonAI image
threshold = 0.35

combined_relabel_map = {}
for (idx, lbl, p) in nonAI_probs_round2:
    if p >= threshold:
        combined_relabel_map[idx] = 5

In [ ]:
#############################################
# Build the final cleaned dataset with all 800 relabeled as AI
#############################################

final_cleaned_train_dataset = CleanedTrainDataset(dataset['train'], transform, combined_relabel_map)

# Check total AI images count
all_labels_after_cleaning = [final_cleaned_train_dataset[i][1] for i in range(len(final_cleaned_train_dataset))]
total_AI = sum([1 for lbl in all_labels_after_cleaning if lbl == 5])
print(f"Total AI images in cleaned dataset: {total_AI}")  # Expecting 800 as per user instruction

Total AI images in cleaned dataset: 770


### Download Dataset

In [ ]:
import csv

# Assuming that final_cleaned_train_dataset[i] returns (image, label)
# and that the underlying 'dataset["train"]' has something like 'image_path' info.
# If not, you need to ensure the dataset or the transform pipeline returns a file identifier.

with open('cleaned_dataset_labels.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['image_path', 'label'])  # Header

    for i in range(len(final_cleaned_train_dataset)):
        img, label = final_cleaned_train_dataset[i]

        # If the original dataset object has a way to retrieve the file name or path:
        image_path = dataset['train'][i]['idx']  # Adjust as needed
        writer.writerow([image_path, label])

print("CSV file with image paths and labels saved as 'cleaned_dataset_labels.csv'.")

In [ ]:
# upload external file before import
from google.colab import files

files.download('cleaned_dataset_labels.csv')

### Train Ming and Qing binary classifier (Failed)

In [53]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np
import random

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Labels: {0: Tang, 1: Song, 2: Yuan, 3: Ming, 4: Qing, 5: AI}
MING_LABEL = 3
QING_LABEL = 4

# Step 1: Filter Ming and Qing images from final_cleaned_train_dataset
ming_qing_indices = []
for i in range(len(final_cleaned_train_dataset)):
    _, label = final_cleaned_train_dataset[i]
    if label in [MING_LABEL, QING_LABEL]:
        ming_qing_indices.append(i)

ming_qing_dataset = Subset(final_cleaned_train_dataset, ming_qing_indices)

# Count Ming and Qing
ming_indices = []
qing_indices = []
for idx in ming_qing_indices:
    _, label = final_cleaned_train_dataset[idx]
    if label == MING_LABEL:
        ming_indices.append(idx)
    else:
        qing_indices.append(idx)

M_count = len(ming_indices)
Q_count = len(qing_indices)
print(M_count/Q_count)
desired_Q = int( M_count/Q_count * M_count)

# Adjust ratio
if Q_count > desired_Q:
    # Too many Qing, downsample Qing
    random.shuffle(qing_indices)
    qing_indices = qing_indices[:desired_Q]
else:
    # Not enough Qing to match 0.63*M, so downsample Ming
    # Q_count should remain as is, and Ming should be adjusted accordingly
    desired_M = int(Q_count / 0.63)
    random.shuffle(ming_indices)
    ming_indices = ming_indices[:desired_M]

# Rebuild the balanced ming-qing dataset
balanced_indices = ming_indices + qing_indices
random.shuffle(balanced_indices)
balanced_dataset = Subset(final_cleaned_train_dataset, balanced_indices)

# Split into train/val (e.g., 80% train, 20% val)
split_ratio = 0.2
val_size = int(len(balanced_indices) * split_ratio)
train_size = len(balanced_indices) - val_size
train_indices, val_indices = balanced_indices[val_size:], balanced_indices[:val_size]

train_subset = Subset(final_cleaned_train_dataset, train_indices)
val_subset = Subset(final_cleaned_train_dataset, val_indices)

# Relabel Ming/Qing to {0: Ming, 1: Qing} for binary classification
class MingQingDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        if label == MING_LABEL:
            new_label = 0
        elif label == QING_LABEL:
            new_label = 1
        else:
            raise ValueError("Label is not Ming or Qing.")
        return img, new_label

train_dataset_mq = MingQingDataset(train_subset)
val_dataset_mq = MingQingDataset(val_subset)

# Initially, assign equal weights to all samples
train_size = len(train_dataset_mq)
weights = np.ones(train_size, dtype=np.float32)
weights = torch.from_numpy(weights)

def create_train_loader(weights):
    # Create a WeightedRandomSampler with current weights
    sampler = torch.utils.data.WeightedRandomSampler(weights, num_samples=len(weights), replacement=True)
    return DataLoader(train_dataset_mq, batch_size=16, sampler=sampler, num_workers=4)

train_loader = create_train_loader(weights)
val_loader = DataLoader(val_dataset_mq, batch_size=16, shuffle=False, num_workers=4)

0.6130884041331802


In [54]:
from copy import deepcopy


# Define a model (ResNet18) for binary classification
model_mq = models.resnet18(pretrained=False)
model_mq.fc = nn.Linear(model_mq.fc.in_features, 2)


model_mq = model_mq.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mq.parameters(), lr=1e-4)

def evaluate(model, loader):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for imgs, lbs in loader:
            imgs, lbs = imgs.to(device), lbs.to(device)
            logits = model(imgs)
            pred = logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
            trues.extend(lbs.cpu().numpy())
    preds = np.array(preds)
    trues = np.array(trues)
    f1 = f1_score(trues, preds, average='macro', zero_division=0)
    cm = confusion_matrix(trues, preds)
    return f1, cm, preds, trues

def evaluate_train_set(model, dataset):
    # Evaluate on train set to find misclassified samples
    # We'll do a regular forward pass (no sampling) just sequential
    loader = DataLoader(dataset, batch_size=16, shuffle=False)
    model.eval()
    misclassified = []
    idx_count = 0
    preds = []
    trues = []
    with torch.no_grad():
        for imgs, lbs in loader:
            imgs, lbs = imgs.to(device), lbs.to(device)
            logits = model(imgs)
            pred = logits.argmax(dim=1)
            correct = (pred == lbs)
            batch_size = imgs.size(0)
            for i in range(batch_size):
                trues.append(lbs[i].item())
                preds.append(pred[i].item())
                if not correct[i]:
                    # record which index was misclassified
                    misclassified.append(idx_count + i)
            idx_count += batch_size
    f1 = f1_score(trues, preds, average='macro', zero_division=0)
    return misclassified, f1

epochs = 15
best_f1 = 0.0
best_model_state = None

for epoch in range(epochs):
    model_mq.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for imgs, lbs in train_loader:
        imgs, lbs = imgs.to(device), lbs.to(device)
        optimizer.zero_grad()
        logits = model_mq(imgs)
        loss = criterion(logits, lbs)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        pred = logits.argmax(dim=1)
        total_correct += (pred == lbs).sum().item()
        total_samples += imgs.size(0)

    avg_loss = total_loss / total_samples
    train_acc = total_correct / total_samples

    # Evaluate on validation set
    val_f1, val_cm, _, _ = evaluate(model_mq, val_loader)

    # Check if this is the best model so far
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model_state = deepcopy(model_mq.state_dict())

    # Print metrics
    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Val F1-score: {val_f1:.4f}")
    print("Val Confusion Matrix:")
    print(val_cm)

    # Focus on misclassified training samples
    misclassified, train_f1 = evaluate_train_set(model_mq, train_dataset_mq)
    print(f"Train F1-score: {train_f1:.4f}, Misclassified samples: {len(misclassified)}")

    # Increase the weights of misclassified samples
    if len(misclassified) > 0:
        weights[misclassified] *= 2.0

    # Recreate train_loader with updated weights
    train_loader = create_train_loader(weights)

    print("-" * 50)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch [1/15]
Train Loss: 0.6331, Train Accuracy: 0.6459
Val F1-score: 0.6622
Val Confusion Matrix:
[[85 30]
 [23 34]]
Train F1-score: 0.6987, Misclassified samples: 199
--------------------------------------------------
Epoch [2/15]
Train Loss: 0.6201, Train Accuracy: 0.6517
Val F1-score: 0.4866
Val Confusion Matrix:
[[37 78]
 [10 47]]
Train F1-score: 0.5788, Misclassified samples: 286
--------------------------------------------------
Epoch [3/15]
Train Loss: 0.5601, Train Accuracy: 0.7199
Val F1-score: 0.5724
Val Confusion Matrix:
[[100  15]
 [ 41  16]]
Train F1-score: 0.6042, Misclassified samples: 227
--------------------------------------------------
Epoch [4/15]
Train Loss: 0.6099, Train Accuracy: 0.6705
Val F1-score: 0.4983
Val Confusion Matrix:
[[38 77]
 [ 9 48]]
Train F1-score: 0.5836, Misclassified samples: 285
--------------------------------------------------
Epoch [5/15]
Train Loss: 0.5758, Train Accuracy: 0.6938
Val F1-score: 0.6458
Val Confusion Matrix:
[[88 27]
 [27 30]

In [49]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import models, transforms
from sklearn.metrics import f1_score, confusion_matrix
import numpy as np
import random
from copy import deepcopy

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Labels: {0: Tang, 1: Song, 2: Yuan, 3: Ming, 4: Qing, 5: AI}
MING_LABEL = 3
QING_LABEL = 4

# Data augmentation: stronger transforms to reduce overfitting
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomResizedCrop(512, scale=(0.8, 1.0)),
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(15),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    # Use the same mean/std as before or compute them from dataset
    transforms.Normalize(dataset_mean,
                         dataset_std)
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(dataset_mean,
                         dataset_std)
])

# Assume final_cleaned_train_dataset is defined and uses no transform yet
# We can wrap it to apply transforms:
class TransformedDataset(Dataset):
    def __init__(self, base_dataset, transform=None):
        self.base_dataset = base_dataset
        self.transform = transform

    def __len__(self):
        return len(self.base_dataset)

    def __getitem__(self, idx):
        img, label = self.base_dataset[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

transformed_final_cleaned = TransformedDataset(final_cleaned_train_dataset, transform=None)

# Step 1: Filter Ming and Qing images
ming_qing_indices = []
for i in range(len(transformed_final_cleaned)):
    _, label = transformed_final_cleaned[i]
    if label in [MING_LABEL, QING_LABEL]:
        ming_qing_indices.append(i)

ming_qing_dataset = Subset(transformed_final_cleaned, ming_qing_indices)

# Count Ming and Qing
ming_indices = []
qing_indices = []
for idx in ming_qing_indices:
    _, label = transformed_final_cleaned[idx]
    if label == MING_LABEL:
        ming_indices.append(idx)
    else:
        qing_indices.append(idx)

M_count = len(ming_indices)
Q_count = len(qing_indices)
desired_Q = int(0.63 * M_count)

# Adjust ratio (downsampling)
if Q_count > desired_Q:
    random.shuffle(qing_indices)
    qing_indices = qing_indices[:desired_Q]
else:
    desired_M = int(Q_count / 0.63)
    random.shuffle(ming_indices)
    ming_indices = ming_indices[:desired_M]

balanced_indices = ming_indices + qing_indices
random.shuffle(balanced_indices)
balanced_dataset = Subset(transformed_final_cleaned, balanced_indices)

# Split train/val
split_ratio = 0.2
val_size = int(len(balanced_indices) * split_ratio)
train_indices, val_indices = balanced_indices[val_size:], balanced_indices[:val_size]

train_subset = Subset(transformed_final_cleaned, train_indices)
val_subset = Subset(transformed_final_cleaned, val_indices)

# Apply transforms separately to train and val subsets
train_dataset_mq = TransformedDataset(train_subset, transform=train_transform)
val_dataset_mq = TransformedDataset(val_subset, transform=val_transform)

class MingQingDataset(Dataset):
    def __init__(self, subset):
        self.subset = subset

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        img, label = self.subset[idx]
        if label == MING_LABEL:
            new_label = 0
        elif label == QING_LABEL:
            new_label = 1
        else:
            raise ValueError("Label is not Ming or Qing.")
        return img, new_label

train_dataset_mq = MingQingDataset(train_dataset_mq)
val_dataset_mq = MingQingDataset(val_dataset_mq)

# No sampler, use entire dataset
train_loader = DataLoader(train_dataset_mq, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset_mq, batch_size=16, shuffle=False)

# Define model
model_mq = models.resnet18(pretrained=False)
# Add dropout before final layer to reduce overfitting:
# One way is to insert dropout after the layer4 or before fc:
# We'll wrap the model to insert dropout:
class ResNetWithDropout(nn.Module):
    def __init__(self, model, p=0.5):
        super().__init__()
        self.features = nn.Sequential(*list(model.children())[:-1]) # everything except fc
        self.fc = nn.Linear(model.fc.in_features, 2)
        self.dropout = nn.Dropout(p=p)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

model_mq = ResNetWithDropout(model_mq, p=0.5).to(device)

# Use weight decay for regularization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_mq.parameters(), lr=1e-4, weight_decay=1e-4)

# Early stopping and LR reduction
patience = 30
no_improve_count = 0
best_f1 = 0.0
best_model_state = None

# Optionally a learning rate scheduler that reduces LR on plateau
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', factor=0.5, patience=2, verbose=True)

def evaluate(model, loader):
    model.eval()
    preds = []
    trues = []
    with torch.no_grad():
        for imgs, lbs in loader:
            imgs, lbs = imgs.to(device), lbs.to(device)
            logits = model(imgs)
            pred = logits.argmax(dim=1).cpu().numpy()
            preds.extend(pred)
            trues.extend(lbs.cpu().numpy())
    preds = np.array(preds)
    trues = np.array(trues)
    f1 = f1_score(trues, preds, average='macro', zero_division=0)
    cm = confusion_matrix(trues, preds)
    return f1, cm

epochs = 30
for epoch in range(epochs):
    model_mq.train()
    total_loss = 0
    total_correct = 0
    total_samples = 0
    for imgs, lbs in train_loader:
        imgs, lbs = imgs.to(device), lbs.to(device)
        optimizer.zero_grad()
        logits = model_mq(imgs)
        loss = criterion(logits, lbs)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        pred = logits.argmax(dim=1)
        total_correct += (pred == lbs).sum().item()
        total_samples += imgs.size(0)

    avg_loss = total_loss / total_samples
    train_acc = total_correct / total_samples

    # Evaluate on validation set
    val_f1, val_cm = evaluate(model_mq, val_loader)

    # Early stopping logic
    if val_f1 > best_f1:
        best_f1 = val_f1
        best_model_state = deepcopy(model_mq.state_dict())
        no_improve_count = 0
    else:
        no_improve_count += 1

    # If no improvement for 'patience' epochs, stop
    if no_improve_count > patience:
        print("Early stopping due to no improvement.")
        break

    # Update LR scheduler
    scheduler.step(val_f1)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {avg_loss:.4f}, Train Accuracy: {train_acc:.4f}")
    print(f"Val F1-score: {val_f1:.4f}")
    print("Val Confusion Matrix:")
    print(val_cm)
    print("-" * 50)

# Load best model state if available
if best_model_state is not None:
    model_mq.load_state_dict(best_model_state)
    print("Loaded best model state from training.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/30]
Train Loss: 0.7421, Train Accuracy: 0.5603
Val F1-score: 0.3725
Val Confusion Matrix:
[[ 7 93]
 [ 0 74]]
--------------------------------------------------
Epoch [2/30]
Train Loss: 0.7062, Train Accuracy: 0.6121
Val F1-score: 0.3946
Val Confusion Matrix:
[[91  9]
 [70  4]]
--------------------------------------------------
Epoch [3/30]
Train Loss: 0.6745, Train Accuracy: 0.6322
Val F1-score: 0.3650
Val Confusion Matrix:
[[100   0]
 [ 74   0]]
--------------------------------------------------
Epoch [4/30]
Train Loss: 0.6575, Train Accuracy: 0.6494
Val F1-score: 0.2984
Val Confusion Matrix:
[[  0 100]
 [  0  74]]
--------------------------------------------------
Epoch [5/30]
Train Loss: 0.6534, Train Accuracy: 0.6595
Val F1-score: 0.3650
Val Confusion Matrix:
[[100   0]
 [ 74   0]]
--------------------------------------------------
Epoch [6/30]
Train Loss: 0.6193, Train Accuracy: 0.6710
Val F1-score: 0.5968
Val Confusion Matrix:
[[56 44]
 [26 48]]
-------------------------

In [ ]:
print(final_cleaned_train_dataset)

In [43]:
torch.save(model_mq.state_dict(), "model_ming_qing.pth")

In [ ]:
from google.colab import files
files.download("model_ming_qing.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>